In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, transforms, Normalize
import torchvision.models as models
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch import optim
from torchsummary import summary
import matplotlib.pyplot as plt
from torchvision.datasets import MNIST, CIFAR10
from torchvision.transforms.functional import pad
import torchvision
from torchtext.datasets import IMDB

from pathlib import Path

import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import gc
gc.collect()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()
print(f"Allocated: {torch.cuda.memory_allocated(device) / (1024 ** 2):.2f} MB")
print(f"Cached: {torch.cuda.memory_reserved(device) / (1024 ** 2):.2f} MB")

Allocated: 0.00 MB
Cached: 0.00 MB


# Getting Started

In [6]:
x=torch.rand(3)
print(x)
type(x)

tensor([0.6375, 0.7364, 0.7428])


torch.Tensor

In [138]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")


Using cuda device


In [139]:
t1=torch.tensor([1,2,3,4,5])
print('1D tensor:',t1)

t2=torch.tensor([[1,2],[3,4],[5,6]])
print('\n2D Tensor:',t2)

t3=torch.tensor([[[1,2],[3,4]],[[5,6],[7,8]]])
print('\n3D Tensor:',t3)

1D tensor: tensor([1, 2, 3, 4, 5])

2D Tensor: tensor([[1, 2],
        [3, 4],
        [5, 6]])

3D Tensor: tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])


In [140]:
t4=torch.rand(2,3)
print('random 2x3 tensor:\n',t4)

t5=torch.zeros(2,3)
print('\n2x3 tensor filled with zeros:\n',t5)

t6=torch.ones(2,3)
print('\n2x3 tensor filled with ones:\n',t6)

t7=torch.empty(2,3)
print('\n2x3 empty tensor:\n',t7)

random 2x3 tensor:
 tensor([[0.0929, 0.8024, 0.5947],
        [0.1535, 0.9415, 0.4627]])

2x3 tensor filled with zeros:
 tensor([[0., 0., 0.],
        [0., 0., 0.]])

2x3 tensor filled with ones:
 tensor([[1., 1., 1.],
        [1., 1., 1.]])

2x3 empty tensor:
 tensor([[ 1.1364e-03, -4.8785e-05,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00]])


In [141]:
t8=torch.tensor([[1,2],[3,4],[5,6]])
print('2D tensor:\n',t8)

print('\nFirst row:',t8[0])

print('\nFirst column:',t8[:,0])

print('\nLast column:',t8[:,-1])

print('\nFirst 2 rows:\n',t8[:2])

2D tensor:
 tensor([[1, 2],
        [3, 4],
        [5, 6]])

First row: tensor([1, 2])

First column: tensor([1, 3, 5])

Last column: tensor([2, 4, 6])

First 2 rows:
 tensor([[1, 2],
        [3, 4]])


In [142]:
t9=torch.tensor([[1,2],[3,4],[5,6]])
t9_reshape=t9.view(2,3)
print('Original 2D tensor:\n',t9)

print('\nReshaped 2D tensor:\n',t9_reshape)

Original 2D tensor:
 tensor([[1, 2],
        [3, 4],
        [5, 6]])

Reshaped 2D tensor:
 tensor([[1, 2, 3],
        [4, 5, 6]])


In [143]:
t10=torch.tensor([[1,2,3],[4,5,6]])
t11=torch.tensor([[10,20,30]])
add=t10+t11
print('Original 2D tensor:\n',t10)

print('\nAdded tensor:\n',add)

matmul=torch.matmul(t10, t10.T)
print('\nMatmul tensor:\n',matmul)

Original 2D tensor:
 tensor([[1, 2, 3],
        [4, 5, 6]])

Added tensor:
 tensor([[11, 22, 33],
        [14, 25, 36]])

Matmul tensor:
 tensor([[14, 32],
        [32, 77]])


In [144]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

tensor_size = (100, 100)  
a = torch.randn(tensor_size, device=device)  
b = torch.randn(tensor_size, device=device)  

c = a + b  


print("Current GPU memory usage:")
print(f"Allocated: {torch.cuda.memory_allocated(device) / (1024 ** 2):.2f} MB")
print(f"Cached: {torch.cuda.memory_reserved(device) / (1024 ** 2):.2f} MB")

Using device: cuda
Current GPU memory usage:
Allocated: 0.12 MB
Cached: 2.00 MB


# Neural Networks

In [2]:
training_data=datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_data=datasets.FashionMNIST(
    root="data",
    train=False, #extracts test data
    download=True,
    transform=ToTensor()
)

batch_size=60
train_dataloader=DataLoader(training_data, batch_size)
test_dataloader=DataLoader(test_data, batch_size)

labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

classes = list(labels_map.values())

## Fully Connected Linear

In [149]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1=nn.Linear(2,4)
        self.fc2=nn.Linear(4,1)
    def forward(self,x):
        x=torch.relu(self.fc1(x))
        x=self.fc2(x)
        return x
model=SimpleNN()
print(model)

SimpleNN(
  (fc1): Linear(in_features=2, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=1, bias=True)
)


In [150]:
X_train=torch.tensor([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
y_train=torch.tensor([[0.0],[1.0],[1.0],[0.0]])

model=SimpleNN()
criterion=nn.MSELoss()
optimizer=optim.SGD(model.parameters(), lr=0.005)


for epoch in range(1000):
    model.train()
    outputs=model(X_train)
    loss=criterion(outputs, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/1000], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 0.3065
Epoch [200/1000], Loss: 0.2618
Epoch [300/1000], Loss: 0.2404
Epoch [400/1000], Loss: 0.2232
Epoch [500/1000], Loss: 0.2077
Epoch [600/1000], Loss: 0.1932
Epoch [700/1000], Loss: 0.1803
Epoch [800/1000], Loss: 0.1678
Epoch [900/1000], Loss: 0.1556
Epoch [1000/1000], Loss: 0.1436


In [151]:
model.eval()
with torch.no_grad():
    test_data=torch.tensor([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
    predictions=model(test_data)
    print(predictions)

tensor([[0.3884],
        [0.6384],
        [0.5996],
        [0.3633]])


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten=nn.Flatten()
        self.linear_relu_stack=nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
        )
    def forward(self, x):
        x=self.flatten(x)
        logits=self.linear_relu_stack(x)
        return logits

model=NeuralNetwork()
print(model)    

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [4]:
loss_fn=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(), lr=0.003)

def train(dataloader, model, loss_fn, optimizer):
    size=len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        pred=model(X)
        loss=loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # if batch % 100 == 0:
    loss, current = loss.item(), (batch + 1) * len(X)
    print(f"loss: {loss}  [{current}/{size}]")
            
def test(dataloader, model, loss_fn):
    size=len(dataloader.dataset)    
    n_batches=len(dataloader)
    model.eval()
    test_loss, correct= 0,0
    with torch.no_grad():
        for X,y in dataloader:
            pred=model(X)
            test_loss+=loss_fn(pred, y).item()
            correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss/=n_batches
    correct/=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [5]:
epochs=10
for i in range(epochs):
    print(f"Epoch {i+1}:\n")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

Epoch 1:

loss: 1.5082645416259766  [60000/60000]
Test Error: 
 Accuracy: 60.0%, Avg loss: 1.473011 

Epoch 2:

loss: 1.0015138387680054  [60000/60000]
Test Error: 
 Accuracy: 66.0%, Avg loss: 0.959777 

Epoch 3:

loss: 0.846994161605835  [60000/60000]
Test Error: 
 Accuracy: 70.0%, Avg loss: 0.803486 

Epoch 4:

loss: 0.7590461373329163  [60000/60000]
Test Error: 
 Accuracy: 73.5%, Avg loss: 0.725632 

Epoch 5:

loss: 0.6968614459037781  [60000/60000]
Test Error: 
 Accuracy: 76.2%, Avg loss: 0.672210 

Epoch 6:

loss: 0.6495867371559143  [60000/60000]
Test Error: 
 Accuracy: 77.9%, Avg loss: 0.631773 

Epoch 7:

loss: 0.6136932373046875  [60000/60000]
Test Error: 
 Accuracy: 79.0%, Avg loss: 0.600784 

Epoch 8:

loss: 0.5865381360054016  [60000/60000]
Test Error: 
 Accuracy: 79.8%, Avg loss: 0.576731 

Epoch 9:

loss: 0.5661325454711914  [60000/60000]
Test Error: 
 Accuracy: 80.6%, Avg loss: 0.557673 

Epoch 10:

loss: 0.5501990914344788  [60000/60000]
Test Error: 
 Accuracy: 81.1%, A

In [6]:
model.eval()
with torch.no_grad():
    for i in range(5):
        X, y = test_data[i][0], test_data[i][1]
        pred = model(X)
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle Boot", Actual: "Ankle Boot"
Predicted: "Pullover", Actual: "Pullover"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Shirt", Actual: "Shirt"


In [13]:
for name, parameter in model.named_parameters():
    print(name, "--",parameter.size())

linear_relu_stack.0.weight -- torch.Size([512, 784])
linear_relu_stack.0.bias -- torch.Size([512])
linear_relu_stack.2.weight -- torch.Size([512, 512])
linear_relu_stack.2.bias -- torch.Size([512])
linear_relu_stack.4.weight -- torch.Size([10, 512])
linear_relu_stack.4.bias -- torch.Size([10])


In [ ]:
# Convolution 1D:
m = nn.Conv1d(16, 33, 3, stride=2)
input = torch.randn(20, 16, 50)
output = m(input)
print(input.size())
print(output.size())
# print(output)
print(m)

torch.Size([20, 16, 50])
torch.Size([20, 33, 24])
Conv1d(16, 33, kernel_size=(3,), stride=(2,))


In [20]:
# Convolution 2D

m1 = nn.Conv2d(16, 33, 3, stride=2)

m2 = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))

m3 = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2), dilation=(3, 1))
input = torch.randn(20, 16, 50, 100)
output1 = m1(input)
output2 = m2(input)
output3 = m3(input)

print(input.size())
print(output1.size())
print(output2.size())
print(output3.size())
# print(output)


torch.Size([20, 16, 50, 100])
torch.Size([20, 33, 24, 49])
torch.Size([20, 33, 28, 100])
torch.Size([20, 33, 26, 100])


In [23]:
# Convolution 3D

m1=nn.Conv3d(16, 33, 2, stride=2)
m2=nn.Conv3d(16, 33, (3, 5, 2), stride=(2, 1, 1), padding=(4, 2, 0))
input=torch.randn(20, 16, 10, 50, 50)
output1=m1(input)
output2=m2(input)
print(input.size())
print(output1.size())
print(output2.size())

torch.Size([20, 16, 10, 50, 50])
torch.Size([20, 33, 5, 25, 25])
torch.Size([20, 33, 8, 50, 49])


In [35]:
#dropout 
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten=nn.Flatten()
        self.linear_relu_stack=nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(512,10)
        )
    def forward(self, x):
        x=self.flatten(x)
        logits=self.linear_relu_stack(x)
        return logits

model=NeuralNetwork()
print(model) 

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [36]:
loss_fn=nn.CrossEntropyLoss()
optimizer=optim.AdamW(model.parameters(), lr=0.001)

def training(dataLoader, model, loss_fn, optimizer):
    size=len(dataLoader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataLoader):
        pred=model(X)
        loss=loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # if batch % 100 == 0:
    loss, current = loss.item(), (batch + 1) * len(X)
    print(f"loss: {loss}  [{current}/{size}]")
            
def testing(dataLoader, model, loss_fn):
    size=len(dataLoader.dataset)    
    n_batches=len(dataLoader)
    model.eval()
    test_loss, correct= 0,0
    with torch.no_grad():
        for X,y in dataLoader:
            pred=model(X)
            test_loss+=loss_fn(pred, y).item()
            correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss/=n_batches
    correct/=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
epochs=10
for i in range(epochs):
    print(f"Epoch {i+1}:\n")
    training(train_dataloader, model, loss_fn, optimizer)
    testing(test_dataloader, model, loss_fn)

Epoch 1:

loss: 0.43239572644233704  [60000/60000]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.451494 

Epoch 2:

loss: 0.38935422897338867  [60000/60000]
Test Error: 
 Accuracy: 85.3%, Avg loss: 0.399360 

Epoch 3:

loss: 0.3372545540332794  [60000/60000]
Test Error: 
 Accuracy: 86.1%, Avg loss: 0.378803 

Epoch 4:

loss: 0.3212348222732544  [60000/60000]
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.354543 

Epoch 5:

loss: 0.36041709780693054  [60000/60000]
Test Error: 
 Accuracy: 87.4%, Avg loss: 0.342899 

Epoch 6:

loss: 0.2732643187046051  [60000/60000]
Test Error: 
 Accuracy: 87.1%, Avg loss: 0.352029 

Epoch 7:

loss: 0.27149611711502075  [60000/60000]
Test Error: 
 Accuracy: 87.3%, Avg loss: 0.341071 

Epoch 8:

loss: 0.3338007628917694  [60000/60000]
Test Error: 
 Accuracy: 87.6%, Avg loss: 0.334360 

Epoch 9:

loss: 0.22312143445014954  [60000/60000]
Test Error: 
 Accuracy: 87.6%, Avg loss: 0.341208 

Epoch 10:

loss: 0.26701176166534424  [60000/60000]
Test Error: 
 Accuracy: 8

In [38]:
model.eval()
with torch.no_grad():
    for i in range(15,20):
        X, y = test_data[i][0], test_data[i][1]
        pred = model(X)
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Trouser", Actual: "Trouser"
Predicted: "Pullover", Actual: "Pullover"
Predicted: "Pullover", Actual: "Coat"
Predicted: "Bag", Actual: "Bag"
Predicted: "T-Shirt", Actual: "T-Shirt"


## CNN

In [57]:
#CNN

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1= nn.Conv2d(1, 64, 2)
        self.conv2= nn.Conv2d(64, 128, 2)
        self.relu= nn.ReLU()
        self.pool= nn.MaxPool2d(2)
        self.fc_shape=None
        self.flat= nn.Flatten()
        self.fc_shape= self.size_tracker(torch.randn(1, 1, 28, 28))
        self.fc= nn.Linear(self.fc_shape, 10)
        
    def size_tracker(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.flat(x)
        return x.shape[1]   
        
    def forward(self, x):
        x= self.pool(self.relu(self.conv1(x)))
        x= self.pool(self.relu(self.conv2(x)))
        x= self.flat(x)
        x= self.fc(x)
        return x

model=CNN()
print(model)
    

CNN(
  (conv1): Conv2d(1, 64, kernel_size=(2, 2), stride=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1))
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=4608, out_features=10, bias=True)
)


In [58]:
loss_fn=nn.CrossEntropyLoss()
optimizer=optim.AdamW(model.parameters(), lr=0.0001)


def training(dataLoader, model, loss_fn, optimizer):
    size=len(dataLoader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataLoader):
        pred=model(X)
        loss=loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # if batch % 100 == 0:
    loss, current = loss.item(), (batch + 1) * len(X)
    print(f"loss: {loss}  [{current}/{size}]")

def testing(dataLoader, model, loss_fn):
    size=len(dataLoader.dataset)    
    n_batches=len(dataLoader)
    model.eval()
    test_loss, correct= 0,0
    with torch.no_grad():
        for X,y in dataLoader:
            pred=model(X)
            test_loss+=loss_fn(pred, y).item()
            correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss/=n_batches
    correct/=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [59]:
epoch=10
for i in range(epoch):
    print(f"Epoch {i+1}:\n")
    training(train_dataloader, model, loss_fn, optimizer)
    testing(test_dataloader, model, loss_fn)

Epoch 1:

loss: 0.5040072798728943  [60000/60000]
Test Error: 
 Accuracy: 81.2%, Avg loss: 0.525819 

Epoch 2:

loss: 0.41093847155570984  [60000/60000]
Test Error: 
 Accuracy: 84.3%, Avg loss: 0.449781 

Epoch 3:

loss: 0.3659093976020813  [60000/60000]
Test Error: 
 Accuracy: 85.6%, Avg loss: 0.415581 

Epoch 4:

loss: 0.33310309052467346  [60000/60000]
Test Error: 
 Accuracy: 86.2%, Avg loss: 0.393739 

Epoch 5:

loss: 0.3093981444835663  [60000/60000]
Test Error: 
 Accuracy: 86.6%, Avg loss: 0.377560 

Epoch 6:

loss: 0.2918350398540497  [60000/60000]
Test Error: 
 Accuracy: 87.1%, Avg loss: 0.365336 

Epoch 7:

loss: 0.27667614817619324  [60000/60000]
Test Error: 
 Accuracy: 87.5%, Avg loss: 0.355653 

Epoch 8:

loss: 0.26431235671043396  [60000/60000]
Test Error: 
 Accuracy: 87.7%, Avg loss: 0.347877 

Epoch 9:

loss: 0.2540619969367981  [60000/60000]
Test Error: 
 Accuracy: 87.9%, Avg loss: 0.341319 

Epoch 10:

loss: 0.2446449249982834  [60000/60000]
Test Error: 
 Accuracy: 88.

In [65]:
model.eval()
with torch.no_grad():
    for i in range(20,25):
        X, y = test_data[i][0], test_data[i][1]
        X = X.unsqueeze(0)
        pred = model(X)
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Pullover", Actual: "Pullover"
Predicted: "Sneaker", Actual: "Sandal"
Predicted: "Sneaker", Actual: "Sneaker"
Predicted: "Sandal", Actual: "Ankle Boot"
Predicted: "Trouser", Actual: "Trouser"


## Transfer Learning

In [9]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')


train_dir= Path("D:\\AppStoneLab\\Pytorch\\data\\pizza_steak_sushi\\train")
test_dir= Path("D:\\AppStoneLab\\Pytorch\\data\\pizza_steak_sushi\\test")

weights= torchvision.models.ResNet50_Weights.DEFAULT

auto_transforms= weights.transforms() #automaticalaly fetches transformations used for resnet dataset
auto_transforms
weights

ResNet50_Weights.IMAGENET1K_V2

In [10]:
train_data = datasets.ImageFolder(train_dir, transform=auto_transforms)
test_data = datasets.ImageFolder(test_dir, transform=auto_transforms)

class_names = train_data.classes

train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader( test_data, batch_size=32, shuffle=False)

# train_dataloader, test_dataloader, class_names
# for X,y in train_dataloader:
#     print(X.shape, y.shape)
#     break

In [11]:
model=models.resnet50(weights=weights).to(device)

summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [12]:
for param in model.parameters():
    param.requires_grad=False
for param in model.fc.parameters():
    param.requires_grad=True

model.fc=nn.Linear(2048, 3)
model.to(device)
summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [18]:
loss_fn= nn.CrossEntropyLoss()
optimizer=optim.AdamW(model.parameters(), lr=0.001)

def training(dataLoader, model, loss_fn, optimizer):
    size=len(dataLoader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataLoader):
        X, y = X.to(device), y.to(device)
        pred=model(X)
        loss=loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss, current = loss.item(), (batch + 1) * len(X)
    print(f"loss: {loss}  [{current}/{size}]")
    
def testing(dataLoader, model, loss_fn):
    size=len(dataLoader.dataset)    
    n_batches=len(dataLoader)
    model.eval()
    test_loss, correct= 0,0
    with torch.no_grad():
        for X,y in dataLoader:
            X, y = X.to(device), y.to(device)
            pred=model(X)
            test_loss+=loss_fn(pred, y).item()

            correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss/=n_batches
    correct/=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [19]:
epoch=20
for i in range(epoch):
    print(f"Epoch {i+1}:\n")
    training(train_dataloader, model, loss_fn, optimizer)
    testing(test_dataloader, model, loss_fn)

Epoch 1:

loss: 1.0324268341064453  [8/225]
Test Error: 
 Accuracy: 89.3%, Avg loss: 0.787780 

Epoch 2:

loss: 1.15571928024292  [8/225]
Test Error: 
 Accuracy: 88.0%, Avg loss: 0.703075 

Epoch 3:

loss: 1.2716748714447021  [8/225]
Test Error: 
 Accuracy: 89.3%, Avg loss: 0.722085 

Epoch 4:

loss: 0.9466521739959717  [8/225]
Test Error: 
 Accuracy: 90.7%, Avg loss: 0.675305 

Epoch 5:

loss: 0.8595020771026611  [8/225]
Test Error: 
 Accuracy: 92.0%, Avg loss: 0.647986 

Epoch 6:

loss: 0.9800182580947876  [8/225]
Test Error: 
 Accuracy: 92.0%, Avg loss: 0.623761 

Epoch 7:

loss: 1.0992772579193115  [8/225]
Test Error: 
 Accuracy: 90.7%, Avg loss: 0.603905 

Epoch 8:

loss: 0.7749406695365906  [8/225]
Test Error: 
 Accuracy: 92.0%, Avg loss: 0.510256 

Epoch 9:

loss: 1.13480806350708  [8/225]
Test Error: 
 Accuracy: 93.3%, Avg loss: 0.560505 

Epoch 10:

loss: 0.9455206394195557  [8/225]
Test Error: 
 Accuracy: 94.7%, Avg loss: 0.509020 

Epoch 11:

loss: 1.2691264152526855  [8/225

In [24]:
model.eval()
with torch.no_grad():
    for i in range(len(test_data)):
        X, y = test_data[i][0], test_data[i][1]
        X = X.unsqueeze(0)
        X = X.to(device)
        pred = model(X)
        predicted, actual = class_names[pred[0].argmax(0)], class_names[y]
        if predicted != actual:
            print(f'at {i}, Predicted: "{predicted}", Actual: "{actual}"')

at 12, Predicted: "steak", Actual: "pizza"
at 29, Predicted: "sushi", Actual: "steak"
at 51, Predicted: "pizza", Actual: "sushi"
at 55, Predicted: "pizza", Actual: "sushi"
at 63, Predicted: "pizza", Actual: "sushi"
at 70, Predicted: "steak", Actual: "sushi"


Partition -------------------- EfficientNet_B0-----------------------------------------

In [ ]:
weights= torchvision.models.EfficientNet_B0_Weights.DEFAULT 

auto_transforms= weights.transforms() #Automaticalaly fetches transformations used for EfficientNet dataset
auto_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [28]:
train_data = datasets.ImageFolder(train_dir, transform=auto_transforms)
test_data = datasets.ImageFolder(test_dir, transform=auto_transforms)

class_names = train_data.classes

train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader( test_data, batch_size=32, shuffle=False)

In [40]:
model= models.efficientnet_b0(weights=weights).to(device)

summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              SiLU-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
              SiLU-6         [-1, 32, 112, 112]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
              SiLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
          Sigmoid-11             [-1, 32, 1, 1]               0
SqueezeExcitation-12         [-1, 32, 112, 112]               0
           Conv2d-13         [-1, 16, 112, 112]             512
      BatchNorm2d-14         [-1, 16, 1

In [41]:
for param in model.parameters():
    param.requires_grad=False
for param in model.classifier.parameters():
    param.requires_grad=True

model.classifier=nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=3, bias=True)
)
model.to(device)
summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              SiLU-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
              SiLU-6         [-1, 32, 112, 112]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
              SiLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
          Sigmoid-11             [-1, 32, 1, 1]               0
SqueezeExcitation-12         [-1, 32, 112, 112]               0
           Conv2d-13         [-1, 16, 112, 112]             512
      BatchNorm2d-14         [-1, 16, 1

In [44]:
loss_fn=nn.CrossEntropyLoss()
optimizer=optim.AdamW(model.parameters(), lr=0.001)

def training(dataLoader, model, loss_fn, optimizer):
    size=len(dataLoader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataLoader):
        X, y= X.to(device), y.to(device)
        pred=model(X)
        loss=loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss, current = loss.item(), (batch+1)*len(X)
    print(f"Loss: {loss}  [{current}/{size}]")
    
def testing(dataLoader, model, loss_fn):
    size=len(dataLoader.dataset)    
    n_batches=len(dataLoader)
    model.eval()
    test_loss, correct= 0,0
    with torch.no_grad():
        for X,y in dataLoader:
            X, y = X.to(device), y.to(device)
            pred=model(X)
            test_loss+=loss_fn(pred, y).item()

            correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss/=n_batches
    correct/=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [45]:
epoch=25
for i in range(epoch):
    print(f"Epoch {i+1}:")
    training(train_dataloader, model, loss_fn, optimizer)
    testing(test_dataloader, model, loss_fn)

Epoch 1:
Loss: 1.3211660385131836  [8/225]
Test Error: 
 Accuracy: 85.3%, Avg loss: 0.707175 

Epoch 2:
Loss: 1.4573718309402466  [8/225]
Test Error: 
 Accuracy: 84.0%, Avg loss: 0.596720 

Epoch 3:
Loss: 1.1044596433639526  [8/225]
Test Error: 
 Accuracy: 84.0%, Avg loss: 0.601943 

Epoch 4:
Loss: 1.3412692546844482  [8/225]
Test Error: 
 Accuracy: 85.3%, Avg loss: 0.535182 

Epoch 5:
Loss: 1.086911678314209  [8/225]
Test Error: 
 Accuracy: 93.3%, Avg loss: 0.548714 

Epoch 6:
Loss: 0.974144458770752  [8/225]
Test Error: 
 Accuracy: 88.0%, Avg loss: 0.490777 

Epoch 7:
Loss: 1.6584638357162476  [8/225]
Test Error: 
 Accuracy: 82.7%, Avg loss: 0.461498 

Epoch 8:
Loss: 1.0076453685760498  [8/225]
Test Error: 
 Accuracy: 86.7%, Avg loss: 0.459437 

Epoch 9:
Loss: 1.379204511642456  [8/225]
Test Error: 
 Accuracy: 82.7%, Avg loss: 0.533686 

Epoch 10:
Loss: 0.9481418132781982  [8/225]
Test Error: 
 Accuracy: 89.3%, Avg loss: 0.471392 

Epoch 11:
Loss: 0.6770046353340149  [8/225]
Test Err

In [46]:
model.eval()
with torch.no_grad():
    for i in range(len(test_data)):
        X, y = test_data[i][0], test_data[i][1]
        X = X.unsqueeze(0)
        X = X.to(device)
        pred = model(X)
        predicted, actual = class_names[pred[0].argmax(0)], class_names[y]
        if predicted != actual:
            print(f'at {i}, Predicted: "{predicted}", Actual: "{actual}"')

at 12, Predicted: "sushi", Actual: "pizza"
at 13, Predicted: "steak", Actual: "pizza"
at 16, Predicted: "sushi", Actual: "pizza"
at 29, Predicted: "pizza", Actual: "steak"
at 43, Predicted: "sushi", Actual: "steak"
at 45, Predicted: "pizza", Actual: "sushi"
at 46, Predicted: "pizza", Actual: "sushi"
at 64, Predicted: "steak", Actual: "sushi"
at 70, Predicted: "steak", Actual: "sushi"


## RNN

In [294]:
#RNN 

df = pd.read_csv(r"D:\\AppStoneLab\\TensorFlow\Datasets\\Clothing-Review.csv")

df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

df.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits


In [295]:
def filter_score(rating):
    return int(rating > 3)

df["Label"] = df["Rating"].apply(filter_score)

df["Label"].value_counts(normalize=True)

Label
1    0.770369
0    0.229631
Name: proportion, dtype: float64

In [296]:
lemm = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def toLower(data):
    if isinstance(data, float):
        return "<UNK>"
    return str(data).lower()

def remove_punctuation_func(text):
    return re.sub(r"[^a-zA-Z0-9\s]", " ", text)

def remove_stopwords(text):
    words = []
    for w in text.split():
        if w not in stop_words:
            words.append(w)
    return " ".join(words)

def lemmatize_sentence(text):
    return " ".join([lemm.lemmatize(w) for w in text.split()])

X = df[["Class Name", "Title", "Review Text", "Label"]].copy()

X["Title"] = X["Title"].apply(toLower).apply(remove_punctuation_func).apply(remove_stopwords).apply(lemmatize_sentence)
X["Review Text"] = X["Review Text"].apply(toLower).apply(remove_punctuation_func).apply(remove_stopwords).apply(lemmatize_sentence)

X["Text"] = X["Title"] + " " + X["Review Text"] + " " + X["Class Name"].astype(str)

X.head()

,Class Name,Title,Review Text,Label,Text
0,Dresses,major design flaw,high hope dress really wanted work initially o...,0,major design flaw high hope dress really wante...
1,Pants,favorite buy,love love love jumpsuit fun flirty fabulous ev...,1,favorite buy love love love jumpsuit fun flirt...
2,Blouses,flattering shirt,shirt flattering due adjustable front tie perf...,1,flattering shirt shirt flattering due adjustab...
3,Dresses,petite,love tracy reese dress one petite 5 foot tall ...,0,petite love tracy reese dress one petite 5 foo...
4,Knits,cagrcoal shimmer fun,aded basket hte last mintue see would look lik...,1,cagrcoal shimmer fun aded basket hte last mint...


In [300]:
from collections import Counter

MAX_VOCAB = 10000   # like TensorFlow tokenizer(num_words=10000)


counter = Counter()

for text in X["Text"]:
    counter.update(text.split())

vocab = ["<PAD>", "<UNK>"]

most_common_words = counter.most_common(MAX_VOCAB - len(vocab))
vocab += [word for word, _ in most_common_words]

vocab_size = len(vocab)

word2index = {word: idx for idx, word in enumerate(vocab)}
index2word = {idx: word for word, idx in word2index.items()}

pad_id = word2index["<PAD>"]
unk_id = word2index["<UNK>"]

print("Vocab size:", vocab_size)
print("PAD id:", pad_id)
print("UNK id:", unk_id)

Vocab size: 10000
PAD id: 0
UNK id: 1


In [ ]:
def pad_features(reviews, pad_id, seq_length=40):
    features = np.full((len(reviews), seq_length), pad_id, dtype=np.int64)

    for i, row in enumerate(reviews):
        row = row[:seq_length]
        features[i, :len(row)] = np.array(row, dtype=np.int64)

    return features

seq_length = 40   

encoding = []
for text in X["Text"]:
    enc = [word2index.get(word, unk_id) for word in text.split()]
    encoding.append(enc)

X["Encoding"] = encoding

# Create padded matrix (numpy array)
X_padded = pad_features(X["Encoding"], pad_id, seq_length)

print("X_padded shape:", X_padded.shape)
print("X_padded dtype:", X_padded.dtype)

X["Padded Encoding"] = list(X_padded)

X_padded shape: (19662, 40)
X_padded dtype: int64


In [302]:
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.25, random_state=100, stratify=y)

trainset = TensorDataset(torch.tensor(X_train, dtype=torch.long),torch.tensor(y_train, dtype=torch.float32))

testset = TensorDataset(torch.tensor(X_test, dtype=torch.long),torch.tensor(y_test, dtype=torch.float32))

train_dataloader = DataLoader(trainset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(testset, batch_size=64, shuffle=False)

len(train_dataloader), len(test_dataloader)

(231, 77)

In [311]:
class RNN(nn.Module):
    def __init__(self, vocab_size, pad_id, embed_dim=128, hidden_dim=64, dropout=0.2):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_id)

        self.rnn1 = nn.RNN(embed_dim, hidden_dim, batch_first=True)
        self.rnn2 = nn.RNN(hidden_dim, hidden_dim, batch_first=True)

        self.fc1 = nn.Linear(hidden_dim, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.embedding(x)         

        out1, _ = self.rnn1(x)         
        out2, h2 = self.rnn2(out1)     

        last_hidden = h2[-1]          

        x = self.fc1(last_hidden)
        x = self.relu(x)
        x = self.dropout(x)

        logits = self.fc2(x).squeeze(1)
        return logits
    
model = RNN(vocab_size=vocab_size, pad_id=pad_id).to(device)
print(model)

RNN(
  (embedding): Embedding(10000, 128, padding_idx=0)
  (rnn1): RNN(128, 64, batch_first=True)
  (rnn2): RNN(64, 64, batch_first=True)
  (fc1): Linear(in_features=64, out_features=128, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)


In [312]:
pos = (y_train == 1).sum()
neg = (y_train == 0).sum()
pos_weight = torch.tensor([neg / pos], dtype=torch.float32).to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# loss_fn=nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def f1_score_binary(y_true, y_pred):
    # y_true, y_pred are tensors of 0/1 floats
    tp = ((y_true == 1) & (y_pred == 1)).sum().item()
    fp = ((y_true == 0) & (y_pred == 1)).sum().item()
    fn = ((y_true == 1) & (y_pred == 0)).sum().item()

    precision = tp / (tp + fp + 1e-8)
    recall    = tp / (tp + fn + 1e-8)

    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    return f1, precision, recall

def training(dataLoader, model, loss_fn, optimizer):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for X, y in dataLoader:
        X, y = X.to(device), y.to(device)

        pred = model(X)  # logits
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * X.size(0)

        probs = torch.sigmoid(pred)
        preds = (probs >= 0.5).float()
        correct += (preds == y).sum().item()
        total += y.size(0)

    avg_loss = total_loss / total
    acc = correct / total
    return avg_loss, acc

def testing(dataLoader, model, loss_fn):
    model.eval()

    total_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_true = []
    with torch.no_grad():
        for X, y in dataLoader:
            X, y = X.to(device), y.to(device)

            logits = model(X)
            loss = loss_fn(logits, y)

            total_loss += loss.item() * X.size(0)

            probs = torch.sigmoid(logits)
            preds = (probs >= 0.5).float()

            correct += (preds == y).sum().item()
            total += y.size(0)

            all_preds.append(preds)
            all_true.append(y)

        avg_loss = total_loss / total
        acc = correct / total

        all_preds = torch.cat(all_preds)
        all_true = torch.cat(all_true)

        f1, precision, recall = f1_score_binary(all_true, all_preds)

        # print(f"Test -> loss: {avg_loss:.4f} | acc: {acc*100:.2f}% | "f"F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}")

    return avg_loss, acc, f1

In [313]:
epochs = 10

for epoch in range(epochs):
    train_loss, train_acc = training(train_dataloader, model, loss_fn, optimizer)
    test_loss, test_acc, test_f1 = testing(test_dataloader, model, loss_fn)

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train -> loss: {train_loss:.4f} | acc: {train_acc*100:.2f}%")
    print(f"Test  -> loss: {test_loss:.4f} | acc: {test_acc*100:.2f}% | F1: {test_f1:.4f}")
    print("-"*50)

Epoch 1/10
Train -> loss: 0.3168 | acc: 55.15%
Test  -> loss: 0.2977 | acc: 64.44% | F1: 0.7282
--------------------------------------------------
Epoch 2/10
Train -> loss: 0.2845 | acc: 70.66%
Test  -> loss: 0.3271 | acc: 45.30% | F1: 0.4969
--------------------------------------------------
Epoch 3/10
Train -> loss: 0.2737 | acc: 70.93%
Test  -> loss: 0.2713 | acc: 78.97% | F1: 0.8614
--------------------------------------------------
Epoch 4/10
Train -> loss: 0.2432 | acc: 78.62%
Test  -> loss: 0.2521 | acc: 79.96% | F1: 0.8667
--------------------------------------------------
Epoch 5/10
Train -> loss: 0.2561 | acc: 73.43%
Test  -> loss: 0.2700 | acc: 73.41% | F1: 0.8129
--------------------------------------------------
Epoch 6/10
Train -> loss: 0.2530 | acc: 78.73%
Test  -> loss: 0.2631 | acc: 76.69% | F1: 0.8418
--------------------------------------------------
Epoch 7/10
Train -> loss: 0.2492 | acc: 77.97%
Test  -> loss: 0.2774 | acc: 74.00% | F1: 0.8194
----------------------

## LSTM

In [ ]:
# LSTM

